# the-stack-yaml-k8s

This is a subset of the stack dataset that only contains valid K8s YAML manifests

In [1]:
! pip install datasets kubernetes-validate PyYAML

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 39.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kubernetes-validate: filename=kubernetes_validate-1.28.0-py3-none-any.whl size=8005656 sha256=0cc5966963d4ef03737ad4de653ee0f498005e8e3be38221e2e3608afea436da
  Stored in directory: /root/.cache/pip/wheels/44/e4/ea/865f36f77c8c76ea045d0f6b375720b534ede31d3ee62c45d3
Successfully built kubernetes-validate


In [2]:
from datasets import load_dataset

ds = load_dataset("bigcode/the-stack", data_dir="data/yaml", split="train")
ds

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 13439939
})

In [23]:
import kubernetes_validate
import yaml

def validate(yaml_content: str):
    try:
        # Speed optimization to return early without having to load YAML
        if "kind" not in yaml_content and "Kind" not in yaml_content:
            return False
        data = yaml.safe_load(yaml_content)
        kubernetes_validate.validate(data, '1.22', strict=True)
        return True
    except Exception as e:
        return False

validate(ds[0]["content"])

False

In [24]:
import os
os.cpu_count()

8

In [25]:
%%time
valid_k8s = ds.select(range(80000)).filter(lambda batch: [validate(x) for x in batch["content"]],
                                           num_proc=os.cpu_count(), batched=True)
valid_k8s

Filter (num_proc=8):   0%|          | 0/80000 [00:00<?, ? examples/s]

CPU times: user 777 ms, sys: 302 ms, total: 1.08 s
Wall time: 1min 44s


Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 1715
})

In [26]:
%%time
valid_k8s = ds.filter(lambda batch: [validate(x) for x in batch["content"]],
                      num_proc=os.cpu_count(), batched=True)

Filter (num_proc=8):   0%|          | 0/13439939 [00:00<?, ? examples/s]

CPU times: user 1min 40s, sys: 11.1 s, total: 1min 51s
Wall time: 4h 40min 29s


In [27]:
valid_k8s

Dataset({
    features: ['hexsha', 'size', 'ext', 'lang', 'max_stars_repo_path', 'max_stars_repo_name', 'max_stars_repo_head_hexsha', 'max_stars_repo_licenses', 'max_stars_count', 'max_stars_repo_stars_event_min_datetime', 'max_stars_repo_stars_event_max_datetime', 'max_issues_repo_path', 'max_issues_repo_name', 'max_issues_repo_head_hexsha', 'max_issues_repo_licenses', 'max_issues_count', 'max_issues_repo_issues_event_min_datetime', 'max_issues_repo_issues_event_max_datetime', 'max_forks_repo_path', 'max_forks_repo_name', 'max_forks_repo_head_hexsha', 'max_forks_repo_licenses', 'max_forks_count', 'max_forks_repo_forks_event_min_datetime', 'max_forks_repo_forks_event_max_datetime', 'content', 'avg_line_length', 'max_line_length', 'alphanum_fraction'],
    num_rows: 276520
})

In [28]:
valid_k8s.push_to_hub("substratusai/the-stack-yaml-k8s")

Pushing dataset shards to the dataset hub:   0%|          | 0/5 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/56 [00:00<?, ?ba/s]